In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv('PS_20174392719_1491204439457_log.csv')

Show that rule based `isFlaggedFraud` did not catch thousands of fradualent transactions

In [3]:
len(raw_data[(raw_data["isFlaggedFraud"] == 0) & (raw_data["isFraud"] == 1)])

8197

Drop the `isFlaggedFraud` since we won't learn anything useful from it.

In [7]:
data = raw_data.drop("isFlaggedFraud", axis=1)
data.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
6362615,743,CASH_OUT,339682.13,C786484425,339682.13,0.0,C776919290,0.00,339682.13,1
6362616,743,TRANSFER,6311409.28,C1529008245,6311409.28,0.0,C1881841831,0.00,0.00,1
6362617,743,CASH_OUT,6311409.28,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,1
6362618,743,TRANSFER,850002.52,C1685995037,850002.52,0.0,C2080388513,0.00,0.00,1
6362619,743,CASH_OUT,850002.52,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,1
